# Step 3-3. 이미지로 유사 이미지 검색
Nova 멀티모달 모델을 사용하여 기준 이미지를 벡터로 변환하고, 유사한 이미지를 검색합니다.

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0

## 1. 설정 (Configuration)

In [24]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
BEDROCK_REGION = _config.get("BEDROCK_REGION", "us-east-1")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

INDEX_NAME = 'nova-image-test'
# TODO: 검색할 이미지 경로를 바꿔보세요!
QUERY_IMAGE_PATH = '../../data/image/bee_1.png'
EMBEDDING_MODEL_ID = 'amazon.nova-2-multimodal-embeddings-v1:0'
VECTOR_DIMENSION = 1024

✅ config.json 로드 완료


## 2. OpenSearch 클라이언트 생성

In [25]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. Bedrock 클라이언트 생성 및 이미지 처리 함수 정의

In [26]:
import json
import base64
import math

session = boto3.Session(profile_name=PROFILE)
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name=BEDROCK_REGION,
)
print("Bedrock client created successfully.")

def normalize_vector(vec):
    """벡터를 L2 정규화합니다."""
    norm = math.sqrt(sum(x * x for x in vec))
    return [x / norm for x in vec] if norm > 0 else vec

def image_to_base64(image_path):
    """이미지 파일 경로를 받아 Base64로 인코딩된 문자열을 반환합니다."""
    if not os.path.exists(image_path):
        print(f"Error: Query image file not found at {image_path}")
        return None
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"Error encoding image {image_path}: {e}")
        return None

def get_image_format(file_name):
    """파일 확장자로 이미지 포맷을 반환합니다."""
    ext = file_name.lower().rsplit('.', 1)[-1]
    return 'png' if ext == 'png' else 'jpeg'

def get_image_embedding_from_bedrock(base64_image_data, image_format, model_id):
    """Nova 멀티모달 모델을 호출하여 이미지의 벡터 임베딩을 반환합니다."""
    body = json.dumps({
        "taskType": "SINGLE_EMBEDDING",
        "singleEmbeddingParams": {
            "embeddingPurpose": "GENERIC_RETRIEVAL",
            "embeddingDimension": VECTOR_DIMENSION,
            "image": {
                "format": image_format,
                "detailLevel": "STANDARD_IMAGE",
                "source": {"bytes": base64_image_data}
            }
        }
    })
    response = bedrock_client.invoke_model(
        body=body,
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
    )
    response_body = json.loads(response.get("body").read())
    return normalize_vector(response_body["embeddings"][0]["embedding"])

Bedrock client created successfully.


In [227]:
# 인덱스에 데이터가 업로드되었는지 확인
if not client.indices.exists(index=INDEX_NAME):
    print(f"❌ '{INDEX_NAME}' 인덱스가 존재하지 않습니다.")
    print("   → Step 3-2b (Nova 이미지 임베딩 데이터 업로드) 노트북을 먼저 실행해주세요.")
else:
    _doc_count = client.count(index=INDEX_NAME)['count']
    if _doc_count == 0:
        print(f"⏳ '{INDEX_NAME}' 인덱스는 있지만 검색 가능한 문서가 0개입니다.")
        print("   → 데이터 업로드 직후라면 인덱싱 중일 수 있습니다. 잠시 후 다시 실행해주세요.")
    else:
        print(f"✅ '{INDEX_NAME}' 인덱스에 {_doc_count}개의 문서가 준비되어 있습니다.")

✅ 'nova-image-test' 인덱스에 6개의 문서가 준비되어 있습니다.


## 4. 이미지 벡터 변환 및 k-NN 검색

In [228]:
from opensearchpy.exceptions import RequestError

print(f"Creating a vector for the query image: '{QUERY_IMAGE_PATH}'...")
base64_data = image_to_base64(QUERY_IMAGE_PATH)
img_format = get_image_format(QUERY_IMAGE_PATH)
query_vector = get_image_embedding_from_bedrock(base64_data, img_format, EMBEDDING_MODEL_ID)

K_NEIGHBORS = 5
RESULT_SIZE = 5

search_query = {
    "size": RESULT_SIZE,
    "query": {
        "knn": {
            "content_vector": {
                "vector": query_vector,
                "k": K_NEIGHBORS
            }
        }
    }
}

print(f"Searching for top {RESULT_SIZE} similar images in index '{INDEX_NAME}'...")

try:
    response = client.search(index=INDEX_NAME, body=search_query)
    hits = response['hits']['hits']
    print(f"\n--- '{QUERY_IMAGE_PATH}'와(과) 유사한 이미지 검색 결과({len(hits)}) ---")

    if not hits:
        print("검색된 이미지가 없습니다.")
    else:
        for i, hit in enumerate(hits):
            score = hit['_score']
            image_path = hit['_source'].get('image_path', 'N/A')
            print(f"\n[{i + 1}] 유사도: {score:.4f}")
            print(f"    이미지 경로: {image_path}")

except RequestError as e:
    print(f"\n[!!!] 검색 중 에러가 발생했습니다.")
    print(f"상태 코드: {e.status_code}")
    print(f"에러 정보: {e.error}")
except Exception as e:
    print(f"\n[!!!] 예상치 못한 에러가 발생했습니다: {e}")

Creating a vector for the query image: '../../data/image/bee_1.png'...
Searching for top 5 similar images in index 'nova-image-test'...

--- '../../data/image/bee_1.png'와(과) 유사한 이미지 검색 결과(5) ---

[1] 유사도: 2.0000
    이미지 경로: ../../data/image/bee_1.png

[2] 유사도: 1.4571
    이미지 경로: ../../data/image/wasp_3.jpg

[3] 유사도: 1.2421
    이미지 경로: ../../data/image/cat_3.jpg

[4] 유사도: 1.1591
    이미지 경로: ../../data/image/dog_3.jpg

[5] 유사도: 1.1516
    이미지 경로: ../../data/image/cat_1.jpg
